## **Load data**

In [97]:
import tensorflow
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7135381678853238874
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6586313605
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11585893254183002879
physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:08:00.0, compute capability: 7.5"
]


In [98]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [99]:
data = pd.read_csv("IMvigor210_clin.csv")
print(data.columns)
print(len(data))

Index(['SampleID', 'Best Confirmed Overall Response', 'binaryResponse',
       'Enrollment IC', 'IC Level', 'TC Level', 'Immune phenotype',
       'FMOne mutation burden per MB', 'Sex', 'Race',
       'Intravesical BCG administered', 'Baseline ECOG Score',
       'Tobacco Use History', 'Met Disease Status', 'Sample age', 'Tissue',
       'Received platinum', 'Sample collected pre-platinum',
       'Neoantigen burden per MB', 'sizeFactor', 'ANONPT_ID', 'os', 'censOS',
       'Lund', 'Lund2', 'TCGA Subtype'],
      dtype='object')
348


In [100]:
marker = pd.read_csv("tmm.csv")
ori_marker = marker 

In [101]:
marker.columns

Index(['Entrez Gene ID', 'Symbol', 'SAMf2ce197162ce', 'SAM698d8d76b934',
       'SAMc1b27bc16435', 'SAM85e41e7f33f9', 'SAMf275eb859a39',
       'SAM7f0d9cc7f001', 'SAM4305ab968b90', 'SAMcf018fee2acd',
       ...
       'SAMe7e4f7c076a7', 'SAMbe25e2c88f3e', 'SAM4caabd64e7fd',
       'SAMc6eff056c89a', 'SAM5cfa1699bdb7', 'SAMda4d892fddc8',
       'SAM3a1c9632ff7b', 'SAM8b4b8b0f9e73', 'SAMe3d4266775a9',
       'SAM2de7cffb5f72'],
      dtype='object', length=350)

## **Data Preprocessing**

In [102]:
marker = marker.rename(columns = {'Symbol':'SampleID'}) 
marker = marker.drop(['Entrez Gene ID'], axis = 1).T
marker.columns = marker.iloc[0]
marker = marker.drop(marker.index[0])

In [103]:
print(marker.head())

SampleID            A1BG      ADA     CDH2     AKT3 POU5F1P5 ZBTB11-AS1  \
SAMf2ce197162ce  3.97148  10.2378  9.12886  10.7241   7.3897    1.97723   
SAM698d8d76b934  6.19179  10.3348  13.7274  11.8855  5.76731    1.77834   
SAMc1b27bc16435  3.56082   9.1993  5.63238  9.97882  6.66971    3.32203   
SAM85e41e7f33f9  3.88137  11.7038  9.27618  11.1349   6.1579    2.14869   
SAMf275eb859a39  4.90625  10.1602  7.49974  9.62024  6.76629    1.88613   

SampleID            MED6    NR2E3  NAALAD2 SNORD116-1  ...  HNRNPDL    DMTF1  \
SAMf2ce197162ce  9.09632   3.4958  5.62838    7.31183  ...  13.3753  11.5898   
SAM698d8d76b934   9.5023  2.08421  4.70671    8.59474  ...  12.5417   12.284   
SAMc1b27bc16435  9.37385   1.4855  2.67817    8.15695  ...  12.9666  11.4569   
SAM85e41e7f33f9  9.15104  2.14869  4.37483    5.17796  ...  12.6481  12.1515   
SAMf275eb859a39  9.12538  3.71523  5.79708    8.12622  ...  13.2014  12.3724   

SampleID          PPP4R1     CDH1  SLC12A6    PTBP3    KCNE2    DGCR

In [104]:
len(marker), len(marker.columns)

(348, 17692)

In [105]:
target_column = 'Immune phenotype'

In [106]:
## match non-digit labels to digits

# data['Best Confirmed Overall Response'] = data['Best Confirmed Overall Response'].map({'CR':0, 'PR':1, 'SD':2, 'PD':3, 'NE':4})
# data['Enrollment IC'] = data['Enrollment IC'].map({'IC0':0, 'IC1':1, 'IC2':2})

# NA = -1 -> nan 빼고 해보기
data['Immune phenotype'] = data['Immune phenotype'].map({'desert':0, 'inflamed':1, 'excluded':2})

# data['Sex'] = data['Sex'].map({'M':0, 'F':1})
# # NA = -1, rm 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER' since the number of it is 1
# data['Race'] = data['Race'].map({'BLACK OF AFRICAN AMERICAN':0, 'ASIAN':1, 'WHITE':2,  'OTHER':4})
# data['Race'] = data['Race'].fillna(-1)
# data['Intravesical BCG administered'] = data['Intravesical BCG administered'].map({'N':0, 'Y':1})
# data['Tobacco Use History'] = data['Tobacco Use History'].map({'NEVER':0, 'PREVIOUS':1, 'CURRENT':2})

# # # replace NA to the mean of N, Y
# # data['Met Disease Status'] = data['Met Disease Status'].map({'Liver':0, 'Visceral':1, 'LN Only':2, 'NA': -1})
# data['Sample age'] = data['Sample age'].map({'(less than) 1 year':0, '1-2 years':1, 'more than 2 years':2})
# data['Received platinum'] = data['Received platinum'].map({'N':0, 'Y':1})

# # replace NA to the mean of N, Y
# #data['Sample collected pre-platinum'] = data['Sample collected pre-platinum'].map({'N':0, 'Y':1, 'NA': 0.5})
# data['Lund'] = data['Lund'].map({'MS1a':0, 'MS1b':1, 'MS2a1':2,'MS2a2':3, 'MS2b1':4, 'MS2b2.1':5, 'MS2b2.2':6})
# data['Lund2'] = data['Lund2'].map({'UroA':0, 'UroB':1, 'Genomically unstable':3, 'Infiltrated':4, 'Basal/SCC-like':5})
# data['TCGA Subtype'] = data['TCGA Subtype'].map({'I':0, 'II':1, 'III':2, 'IV':3})     

# # replace NA to the mean of column 
# data['Neoantigen burden per MB'] = data['Neoantigen burden per MB'].fillna(data.mean()['Neoantigen burden per MB'])
# # replace NA to the mean of column 
# data['FMOne mutation burden per MB'] = data['FMOne mutation burden per MB'].fillna(data.mean()['FMOne mutation burden per MB'])

  

print(data[target_column])


0      NaN
1      2.0
2      2.0
3      1.0
4      2.0
      ... 
343    NaN
344    NaN
345    2.0
346    NaN
347    NaN
Name: Immune phenotype, Length: 348, dtype: float64


In [107]:
if target_column in marker.columns:
  marker = marker.drop(target_column, axis = 1)

In [108]:
data.index = data['SampleID']
marker.index.equals(data.index)

True

In [109]:
marker = marker.join(data[target_column])
marker = marker.reset_index()
marker = marker.rename(columns = {'index':'SampleID'}) 
marker.head()

,SampleID,A1BG,ADA,CDH2,AKT3,POU5F1P5,ZBTB11-AS1,MED6,NR2E3,NAALAD2,...,DMTF1,PPP4R1,CDH1,SLC12A6,PTBP3,KCNE2,DGCR2,CASP8AP2,SCO2,Immune phenotype
0,SAMf2ce197162ce,3.97148,10.2378,9.12886,10.7241,7.3897,1.97723,9.09632,3.4958,5.62838,...,11.5898,12.1645,15.7093,12.7483,13.4474,2.61833,9.76685,11.1387,10.2317,NaN
1,SAM698d8d76b934,6.19179,10.3348,13.7274,11.8855,5.76731,1.77834,9.5023,2.08421,4.70671,...,12.284,11.7372,13.5991,11.7458,12.0886,3.05154,9.8475,11.4952,10.4029,2.0
2,SAMc1b27bc16435,3.56082,9.1993,5.63238,9.97882,6.66971,3.32203,9.37385,1.4855,2.67817,...,11.4569,11.3791,15.5232,11.8236,13.9629,0,9.9633,10.9028,9.24281,2.0
3,SAM85e41e7f33f9,3.88137,11.7038,9.27618,11.1349,6.1579,2.14869,9.15104,2.14869,4.37483,...,12.1515,12.3973,15.1202,13.0617,12.9823,2.62091,9.56163,11.8,11.1349,1.0
4,SAMf275eb859a39,4.90625,10.1602,7.49974,9.62024,6.76629,1.88613,9.12538,3.71523,5.79708,...,12.3724,12.4545,15.2666,13.831,13.0864,2.12781,10.2516,11.416,10.5975,2.0


In [110]:
marker.index

RangeIndex(start=0, stop=348, step=1)

In [111]:
marker = marker.dropna(axis=0)
marker.shape

(284, 17694)

In [112]:
marker.reset_index(inplace=True, drop=True)
marker.index

RangeIndex(start=0, stop=284, step=1)

In [113]:
######17694 check
marker.shape

(284, 17694)

In [114]:
x_train_cls0_list, x_train_cls1_list, x_train_cls2_list = [], [], []
df_x_train_cls0, df_x_train_cls1, df_x_train_cls2 = pd.DataFrame(columns = marker.columns), pd.DataFrame(columns = marker.columns), pd.DataFrame(columns = marker.columns)

In [115]:
# split train and test data
# 10-fold cross validation

train_list = []
test_list = []

from sklearn.model_selection import StratifiedKFold
# 10-fold cv
nsplit = 10
fold = StratifiedKFold(n_splits=nsplit, shuffle=False, random_state=0)

C:\Users\user\anaconda3\envs\imvigor_keras\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [116]:
for train_idx, test_idx in fold.split(marker, marker[target_column]):
    # print("TRAIN:", train_idx, "\nTEST:", test_idx)
    train_list.append(marker.loc[train_idx])
    test_list.append(marker.loc[test_idx])

    train_df = marker.loc[train_idx]
    df_x_train_cls0 = np.asarray(train_df.loc[train_df[target_column].isin([0.0])].drop(['SampleID', 'Immune phenotype'], axis=1)).astype(np.float32)
    df_x_train_cls1 = np.asarray(train_df.loc[train_df[target_column].isin([1.0])].drop(['SampleID', 'Immune phenotype'], axis=1)).astype(np.float32)
    df_x_train_cls2 = np.asarray(train_df.loc[train_df[target_column].isin([2.0])].drop(['SampleID', 'Immune phenotype'], axis=1)).astype(np.float32)

    x_train_cls0_list.append(df_x_train_cls0)
    x_train_cls1_list.append(df_x_train_cls1)
    x_train_cls2_list.append(df_x_train_cls2)


print(len(train_list), len(test_list))

10 10


In [117]:
# groupby 1 df

print(train_list[0].groupby(target_column).size())
print(test_list[0].groupby(target_column).size()) 

Immune phenotype
0.0     68
1.0     67
2.0    120
dtype: int64
Immune phenotype
0.0     8
1.0     7
2.0    14
dtype: int64


In [118]:
len(train_list)

10

In [119]:
# # check the average number of total 5 training data per class
# total_train = pd.concat([train_list[0], train_list[1], train_list[2], train_list[3], train_list[4]], axis=0, ignore_index=True)
# print(total_train.groupby(target_column).size())
# print('\n', total_train.groupby(target_column).size()/5)

In [120]:
# # check the average number of total 5 test data per class
# total_test = pd.concat([test_list[0], test_list[1], test_list[2], test_list[3], test_list[4]], axis=0, ignore_index=True)
# print(total_test.groupby(target_column).size())
# print('\n', total_test.groupby(target_column).size()/5)

In [121]:
from sklearn.preprocessing import LabelEncoder
encoder =  LabelEncoder()

In [122]:
x_train_list, x_test_list, y_train_list, y_test_list = [], [], [], []
decoded_y_train_list, decoded_y_test_list = [], []

In [123]:
for (train, test) in zip(train_list, test_list): 
  decoded_y_train = train[target_column]
  Y = encoder.fit_transform(decoded_y_train)
  y_train = pd.get_dummies(Y).values

  decoded_y_test = test[target_column]
  Y = encoder.fit_transform(decoded_y_test)
  y_test = pd.get_dummies(Y).values

  x_train = train.drop([target_column], axis = 1)
  x_test = test.drop([target_column], axis = 1)

  features = marker.columns.tolist()
  features.remove(target_column)
  features.remove('SampleID')

  x_train.set_index(x_train['SampleID'], inplace=True, drop=True)
  x_test.set_index(x_test['SampleID'], inplace=True, drop=True)

  x_train.drop(['SampleID'], axis=1, inplace=True)
  x_test.drop(['SampleID'], axis=1, inplace=True)
  x_train.reset_index(drop=True, inplace=True)
  x_test.reset_index(drop=True, inplace=True)


  x_test = np.asarray(x_test).astype(np.float32)
  x_train = np.asarray(x_train).astype(np.float32)

  x_train_list.append(x_train)
  x_test_list.append(x_test)
  y_train_list.append(y_train)
  y_test_list.append(y_test)
  decoded_y_train_list.append(decoded_y_train)
  decoded_y_test_list.append(decoded_y_test)

C:\Users\user\anaconda3\envs\imvigor_keras\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [124]:
from keras import backend as K
import tensorflow as tf

def result(one_pred, y_test):
  acc, rec, pre = [], [], []
  
  """total Accuracy"""
  total_acc = K.sum(one_pred[:,:] * y_test[:,:])/y_test.shape[0]

  for target_class in range(y_test.shape[1]):
    """RECALL per class"""

    tp = K.sum(one_pred[:, target_class] * y_test[:, target_class])  #dtype=float32
    tpfn = tf.cast(K.sum(y_test[:, target_class]), tf.float32)  #uint8 to float32

    # Recall =  (True Positive) / (True Positive + False Negative)
    recall = tp / (tpfn + K.epsilon())

    """PRECISION per class"""

    tpfp = tf.cast(K.sum(one_pred[:, target_class]), tf.float32)

    # Precision = (True Positive) / (True Positive + False Positive)
    precision = tp / (tpfp + K.epsilon())

    """Accuracy per class"""
    tn = np.sum((one_pred[:, target_class]==0) * (y_test[:, target_class]==0).T)

    accuracy = (tp + tn)/(tpfn + tpfp - tp + tn)

    acc.append(accuracy)
    rec.append(recall)
    pre.append(precision)

    # print("\nclass: ", target_class)
    # print("tp: ", tp)
    # print("tp+fn: ", tpfn)
    # print("tp+fp: ", tpfp)

    # tf.print("Accuracy = {:.3f}".format(accuracy))
    # tf.print("Recall = {:.3f}".format(recall))
    # tf.print("Precision = {:.3f} ".format(precision))

  return acc, rec, pre, total_acc

# **T-Test**


In [125]:
from statsmodels.sandbox.stats.multicomp import MultiComparison
import scipy.stats

In [126]:
x_train.shape[1]

17692

In [127]:
x_train_cls0_list[0].shape

(68, 17692)

In [128]:
x_train_cls0_list[0][0]

array([ 5.316224 ,  7.6651883,  6.9541984, ..., 10.070971 , 10.705562 ,
        9.4054575], dtype=float32)

In [129]:
df_x = marker.drop(['SampleID', target_column], axis = 1)
df_x.head()

,A1BG,ADA,CDH2,AKT3,POU5F1P5,ZBTB11-AS1,MED6,NR2E3,NAALAD2,SNORD116-1,...,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,PTBP3,KCNE2,DGCR2,CASP8AP2,SCO2
0,6.19179,10.3348,13.7274,11.8855,5.76731,1.77834,9.5023,2.08421,4.70671,8.59474,...,12.5417,12.284,11.7372,13.5991,11.7458,12.0886,3.05154,9.8475,11.4952,10.4029
1,3.56082,9.1993,5.63238,9.97882,6.66971,3.32203,9.37385,1.4855,2.67817,8.15695,...,12.9666,11.4569,11.3791,15.5232,11.8236,13.9629,0,9.9633,10.9028,9.24281
2,3.88137,11.7038,9.27618,11.1349,6.1579,2.14869,9.15104,2.14869,4.37483,5.17796,...,12.6481,12.1515,12.3973,15.1202,13.0617,12.9823,2.62091,9.56163,11.8,11.1349
3,4.90625,10.1602,7.49974,9.62024,6.76629,1.88613,9.12538,3.71523,5.79708,8.12622,...,13.2014,12.3724,12.4545,15.2666,13.831,13.0864,2.12781,10.2516,11.416,10.5975
4,5.12333,8.82476,7.47552,12.0178,6.72309,2.48973,9.33838,2.02779,9.2093,8.29892,...,12.7182,12.6469,12.3837,14.568,12.6963,12.953,3.55733,9.2681,11.5961,10.6172


In [130]:
# all column names
col_name = df_x.columns.tolist()
len(col_name)

17692

In [131]:
from scipy import stats

pval_list = []

def bonferroni_correction_function(cls0, cls1, alpha, no_test):
    # 2.8261361067148994e-06
    alpha_bonferroni = alpha/no_test 
    
    # number of valid features
    counter = 0
    lowpval= []
    for i in range(no_test):
        statistic, pvalue = stats.ttest_ind(cls0[i], cls1[i], equal_var=False)
        # print(len(statistic)) #17962
        # print(len(pvalue)) #17692
        # print(pvalue)

        # low p-value
        if pvalue <= alpha_bonferroni:
            lowpval.append(pvalue)
            counter = counter + 1

            # size (selected feature count)*(number of training data per class) array
            features.append(i)

        # break
    # class0, class1의 경우 대략 900 ~ 1300개
    # class1, class2의 경우 대략 200 ~ 300개
    # class2, class0의 경우 대략 200 ~ 400개
    # print("number of selected features per fold: ", counter)

    # save feature index without duplication
    # print("# of features w dups: ", len(features))
    
    nodup_features = list(set(features))
    # print("# of features w/o dups: ", len(nodup_features))

    return nodup_features, lowpval

In [132]:
# 'desert':0, 'inflamed':1, 'excluded':2  
# no_test=17692
# compare an array of each class

pval0, pval1, pval2 = [], [], []
# selected feature indexes with low p-value
feature_list = []

if x_train.shape[1] == 17692: 
  for cls0, cls1, cls2 in zip(x_train_cls0_list, x_train_cls1_list, x_train_cls2_list):

    features = []

    # print("\ncls0, cls1")
    _, pv0 = bonferroni_correction_function(cls0.T, cls1.T, alpha=0.05, no_test=x_train.shape[1])
    # print("\ncls1, cls2")
    _, pv1 = bonferroni_correction_function(cls1.T, cls2.T, alpha=0.05, no_test=x_train.shape[1])
    # print("\ncls2, cls0")
    final_nodup_features, pv2 = bonferroni_correction_function(cls2.T, cls0.T, alpha=0.05, no_test=x_train.shape[1])

    pval0.append(pv0)
    pval1.append(pv1)
    pval2.append(pv2)

    # Feature indexes per fold
    feature_list.append(final_nodup_features)

In [133]:
col_name_list = []
# {name:index} pairs of selected feature from x_train_cls_list
col_idx_map_list = []

for i, (cls0, cls1, cls2) in enumerate(zip(x_train_cls0_list, x_train_cls1_list, x_train_cls2_list)):
  col_name_list.append(np.array(col_name)[feature_list[i]].astype(np.str))  

  col_idx_dict = dict(zip(feature_list[i], col_name_list[i]))
  col_idx_map_list.append(col_idx_dict)

  # cls with important features
  x_train_cls0_list[i] = cls0[:, feature_list[i]]
  x_train_cls1_list[i] = cls1[:, feature_list[i]]
  x_train_cls2_list[i] = cls2[:, feature_list[i]]

In [134]:
import collections

for i, pair in enumerate(col_idx_map_list): 
    col_idx_map_list[i] =  collections.OrderedDict(sorted(pair.items()))
    
col_idx_map_list[0]

OrderedDict([(1, 'ADA'),
             (58, 'MIR29B2CHG'),
             (80, 'MKRN2OS'),
             (164, 'FCGR1CP'),
             (171, 'IPO5P1'),
             (201, 'SH2B3'),
             (225, 'ANAPC1P1'),
             (236, 'LINC00173'),
             (239, 'LOC100287896'),
             (272, 'nan'),
             (305, 'IGLL5'),
             (309, 'MAMLD1'),
             (342, 'SMIM25'),
             (371, 'SLFN12L'),
             (400, 'VIM-AS1'),
             (408, 'PSMB8-AS1'),
             (419, 'nan'),
             (441, 'KLRC4-KLRK1'),
             (466, 'ABCC5'),
             (472, 'TPTE2P5'),
             (490, 'IL18BP'),
             (497, 'PTPRU'),
             (498, 'TSPAN32'),
             (543, 'ANTXRLP1'),
             (554, 'nan'),
             (596, 'nan'),
             (613, 'RASGRP1'),
             (617, 'FRY'),
             (630, 'FAM13A'),
             (633, 'EBI3'),
             (640, 'PLXNC1'),
             (738, 'LOC101928548'),
             (774, 'nan'),
   

In [135]:
len(col_idx_map_list[0])

1241

In [136]:
col_name_list[0]
# col_name_list is unsorted!!

array(['ADA', 'MTMR3', 'AP1S2', ..., 'FAM174B', 'SPHK1', 'GBP1P1'],
      dtype='<U23')

In [137]:
# check nan
for c in col_name_list:
    print(c.shape, [True for name in c if name=="nan"])

(1241,) [True, True, True, True, True, True, True, True, True, True, True, True]
(1205,) [True, True, True, True, True, True, True, True, True]
(1292,) [True, True, True, True, True, True, True, True, True, True]
(1272,) [True, True, True, True, True, True, True, True, True, True, True, True]
(1223,) [True, True, True, True, True, True, True, True, True]
(993,) [True, True, True, True, True, True, True, True, True]
(1224,) [True, True, True, True, True, True, True, True, True, True]
(1342,) [True, True, True, True, True, True, True, True, True, True]
(1173,) [True, True, True, True, True, True, True, True, True, True, True]
(953,) [True, True, True, True, True, True, True, True, True, True]


In [138]:
x_train_cls2_list[0].shape

(120, 1241)

In [139]:
# 약 10*1100개
print(len(pval0), len(pval1), len(pval2))

10 10 10


In [140]:
for p in pval0:
    print(len(p))

1209
1176
1263
1253
1208
965
1188
1330
1145
935


In [141]:
all_pval0 = [item for sublist in pval0 for item in sublist]
all_pval1 = [item for sublist in pval1 for item in sublist]
all_pval2 = [item for sublist in pval2 for item in sublist]

In [142]:
print(np.mean(all_pval0))
print(np.mean(all_pval1))
print(np.mean(all_pval2))
print('\n')
print(np.std(all_pval0))
print(np.std(all_pval1))
print(np.std(all_pval2))

3.297125217763904e-07
3.931670534638763e-07
4.271195302021916e-07


6.408081679404127e-07
6.800006258506809e-07
6.952008495830517e-07


In [143]:
for i in range(len(feature_list)):
  # selected number of features per fold
  print(len(feature_list[i]))

1241
1205
1292
1272
1223
993
1224
1342
1173
953


In [144]:
for i, (x_train, x_test, y_train, y_test) in enumerate(zip(x_train_list, x_test_list, y_train_list, y_test_list)):
  x_train = x_train[:, feature_list[i]]
  x_test = x_test[:, feature_list[i]]
    
  x_train_list[i] = x_train
  x_test_list[i] = x_test
  y_train_list[i] = y_train
  y_test_list[i] = y_test

  # (, 900 ~ 1300)
  print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(255, 1241) (29, 1241) (255, 3) (29, 3)
(255, 1205) (29, 1205) (255, 3) (29, 3)
(255, 1292) (29, 1292) (255, 3) (29, 3)
(255, 1272) (29, 1272) (255, 3) (29, 3)
(256, 1223) (28, 1223) (256, 3) (28, 3)
(256, 993) (28, 993) (256, 3) (28, 3)
(256, 1224) (28, 1224) (256, 3) (28, 3)
(256, 1342) (28, 1342) (256, 3) (28, 3)
(256, 1173) (28, 1173) (256, 3) (28, 3)
(256, 953) (28, 953) (256, 3) (28, 3)


# **Neural Networks**

In [145]:
y_all_test, y_all_proba = [], []

In [146]:
one_pred_list, train_acc_list, total_acc_list = [], [], []
w_list = []

In [147]:
n_classes = y_train.shape[1]

In [148]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import LeakyReLU

from keras.optimizers import Adam
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import BatchNormalization, Activation

import tensorflow as tf
import datetime

from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_recall_curve, roc_curve, auc
from sklearn.preprocessing import label_binarize

from operator import add

start = datetime.datetime.now()
cls = ['desert', 'inflamed', 'excluded']

%matplotlib inline

EarlyStopping = EarlyStopping(monitor='accuracy', patience=1000, verbose=1)

epoch = 2500
h_loss, h_val_loss, h_acc, h_val_acc = [0]*epoch, [0]*epoch, [0]*epoch, [0]*epoch

reg =tf.keras.regularizers.l1(0.01)
reg2 =tf.keras.regularizers.l1(0.08)

for idx, (x_train, x_test, y_train, y_test) in enumerate(zip(x_train_list, x_test_list, y_train_list, y_test_list)):
#   mc = ModelCheckpoint('best_model_'+str(idx)+'.h5', monitor='val_loss', mode='min', save_best_only=True)
    
  with tf.device('/gpu:0'):
  
    model = Sequential()
    # input_shape=(x_train.shape[1],), activation='relu'))
    #  kernel_regularizer=reg
#     model.add(Dense(32, input_shape=(x_train.shape[1],), activation='relu'))

#     model.add(Dropout(0.3))
#     model.add(Dense(32, activation='relu'))


#     model.add(Dense(y_train.shape[1], activation='softmax'))

    model.add(Dense(y_train.shape[1], input_shape=(x_train.shape[1],), kernel_regularizer=reg2))

    opt = Adam(learning_rate=0.00001)
    model.compile(loss='categorical_crossentropy', 
                  optimizer=opt, 
                  metrics=['accuracy']) 

    hist = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epoch, batch_size=128, verbose=0)
    h_loss = list(map(add, h_loss, hist.history['loss']))
    h_val_loss = list(map(add, h_val_loss, hist.history['val_loss']))
    h_acc = list(map(add, h_acc, hist.history['accuracy']))
    h_val_acc = list(map(add, h_val_acc, hist.history['val_accuracy']))

  
  
#   plt.subplot(1,2,1)
#   plt.figure(figsize=(12,8))
#   plt.plot(hist.history['accuracy'])
#   plt.plot(hist.history['val_accuracy'])
#   # plt.legend(['training loss','test loss', 'training accuracy','test accuracy'])
#   plt.legend(['training accuracy','test accuracy'], prop={'size': 26})
#   plt.grid()
#   plt.show()

#   plt.subplot(1,2,2)
#   plt.figure(figsize=(10,5))
#   plt.plot(hist.history['loss'])
#   plt.plot(hist.history['val_loss'])
#   plt.legend(['training loss','test loss'])
#   plt.grid() 
#   plt.show()


  # training accuracy
  # print("Train Accuracy: ", hist.history['accuracy'][-1])
  train_acc_list.append(hist.history['accuracy'][-1])

  # test accuracy
  loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
  # print("Test Accuracy = {:.3f}".format(accuracy))
  print(round(accuracy, 3))
  total_acc_list.append(accuracy)
  

  prediction = model.predict(x_test, verbose=0)
  y_all_test.append(y_test)
  y_all_proba.append(prediction)
  # print(prediction)
  
  one_pred = np.zeros_like(prediction)
  # one hot encoding of prediction
  one_pred[np.arange(len(prediction)), prediction.argmax(1)] = 1
  one_pred_list.append(one_pred)
    
  weights = model.layers[0].get_weights()
  w = weights[0]
  w_list.append(w)



end = datetime.datetime.now()
time_delta = end - start

0.379
0.483
0.241
0.621
0.25
0.393
0.464
0.25
0.107
0.321


In [149]:
tf.print("Total test Accuracy = {:.3f}".format(K.sum(total_acc_list)/len(total_acc_list)))

Total test Accuracy = 0.351


In [150]:
for w in w_list:
    print(w.shape)
#     print(w)

(1241, 3)
(1205, 3)
(1292, 3)
(1272, 3)
(1223, 3)
(993, 3)
(1224, 3)
(1342, 3)
(1173, 3)
(953, 3)


In [151]:
ids_list = []
n = 20
for w in w_list:
  feature_w = np.sum(np.absolute(w), axis=1)
  ids = feature_w.argsort()[::-1][:n]
  ids_list.append(ids)
#   print(w)
  # ids

In [152]:
for ids in ids_list:
    print(ids)

[ 743  837  401  300   22 1155  995  628  754 1194  792 1167  454   50
  440  458  534  317   94 1224]
[ 904 1000  733  994  329  642  493   40  346 1106  719  160  538  724
 1093  857  323  638  286  960]
[ 574  799  628 1226  492  302  745  507  652  548  598  264    3  124
 1077 1056  596 1166  226  130]
[ 138  228  244  750 1025  190  406  426  823  264  684 1151 1136  463
 1253  950  724 1127  195  398]
[ 663 1005  742  395  937  764  337   20  310  302  164  373  498  402
   52  106  394  503  476  737]
[753 615 472 538  93 527 151 934   3 247 276 841 510 497 808 245 696 478
 425 209]
[282 478 651 752 277 816 182 329 953 509 830 590 577 800 945 941  86 850
 631 705]
[ 343  711 1336   30  787  580  634  294  598 1309 1057  599  172  295
  414  761  441  517 1085  359]
[ 488  305  586  558  516  495  359  807  420   66  345  745  174  714
  769  338  121  959   55 1075]
[815 383 652 888  62 752 351 733 201 577 521 204 925 582 446 778 767 575
 371 920]


In [153]:
# get index of original training feature table
top_feature_idx_list = []

for i, ids in enumerate(ids_list):
    top_feature_idx = np.array(list(col_idx_map_list[i]))[ids]
    top_feature_idx_list.append(top_feature_idx)
    
top_feature_idx_list

[array([10159, 11879,  6346,  4583,   498, 16856, 14306,  8602, 10358,
        17177, 11070, 16938,  7009,  1056,  6964,  7013,  7628,  4927,
         1711, 17475]),
 array([13332, 15050, 10435, 15008,  5143,  9030,  7514,  1003,  5415,
        16771, 10178,  2771,  7755, 10259, 16601, 12559,  5122,  8960,
         4422, 14420]),
 array([ 7755, 10680,  8335, 17066,  7157,  4424,  9852,  7297,  8556,
         7569,  7991,  3956,   171,  2151, 15042, 14564,  7965, 16418,
         3379,  2222]),
 array([ 2302,  3472,  3683, 10068, 14433,  3007,  6086,  6404, 11384,
         3949,  9064, 16454, 16250,  6955, 17475, 13269,  9753, 16119,
         3131,  5990]),
 array([ 9165, 14784, 10278,  6254, 13655, 10775,  5231,   497,  4778,
         4575,  2771,  5834,  7452,  6346,  1096,  2004,  6206,  7517,
         7243, 10180]),
 array([13329, 10303,  8036,  8945,  1959,  8748,  3002, 16999,   164,
         4568,  5251, 15266,  8536,  8335, 14483,  4488, 12208,  8085,
         7569,  3937]),
 arr

In [154]:
list(col_idx_map_list[0])[1]
# list(col_idx_map_list[0].values())[1]

58

In [155]:
top_feature_name_list = []

for i, ids in enumerate(ids_list):
    top_feature_name = np.array(list(col_idx_map_list[i].values()))[ids]
    top_feature_name_list.append(top_feature_name)
# top_feature_name_list  

In [156]:
# check nan, print True if name contains 'nan'
for t in top_feature_name_list:
    print(t.shape, [True for name in t if name=="nan"])

(20,) []
(20,) []
(20,) []
(20,) []
(20,) []
(20,) []
(20,) []
(20,) []
(20,) []
(20,) []


In [157]:
seen = set()
repeated = set()

# check repeated value by original index
for feature_idx in top_feature_idx_list:
  for i in set(feature_idx):
    if i in seen:
      repeated.add(i)
    else:
      seen.add(i)
    
repeated_idx = list(repeated)
repeated_idx

[14784,
 7297,
 17475,
 10180,
 6346,
 7755,
 9165,
 8335,
 7569,
 12146,
 2771,
 4243,
 10259,
 7288,
 7452,
 10303]

In [158]:
# non bonf 
# seen = set()
# repeated = set()

# # check repeated value by original index
# for idx in ids_list:
#   for i in set(idx):
#     if i in seen:
#       repeated.add(i)
#     else:
#       seen.add(i)
    
# repeated_idx = list(repeated)
# repeated_idx

In [159]:
repeated_name = np.array(col_name)[repeated_idx]
repeated_name

array(['TMEM156', 'ARSI', 'TOX', 'XAF1', 'MILR1', 'SPATC1', 'FOXP3',
       'ARRB2', 'TNFRSF9', 'RNASE6', 'DBH-AS1', 'FABP6', 'TENT5C', 'ID3',
       'APOE', 'LAX1'], dtype='<U23')

In [160]:
repeated_name[0] in col_idx_map_list[0].values()

True

In [161]:
# check all repeated_features exist in selected feature dictionary per fold
# [True if name in col_idx_map.values() else False for col_idx_map in col_idx_map_list for name in repeated_name]

for col_idx_map in col_idx_map_list:
    print(["T" if name in col_idx_map.values() else False for name in repeated_name])

['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', False, 'T', 'T', 'T', 'T']
['T', 'T', 'T', 'T', False, 'T', 'T', 'T', 'T', 'T', 'T', False, 'T', 'T', 'T', 'T']
['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
['T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T']
['T', False, 'T', 'T', False, 'T', 'T', 'T', 'T', 'T', 'T', False, 'T', 'T', 'T', 'T']


In [162]:
r_name = repeated_name.tolist()
# r_idx = repeated_idx.tolist()

rm_list = []
for col_idx_map in col_idx_map_list:
    for i, name in enumerate(repeated_name):
        if name not in col_idx_map.values():
            rm_list.append(i)
            
rm_list = list(dict.fromkeys(rm_list))            
for i in rm_list:
    r_name.pop(i)
    repeated_idx.pop(i)

repeated_name = np.array(r_name)
# repeated_idx = np.array(r_idx)

In [163]:
# final 
repeated_name

array(['TMEM156', 'TOX', 'XAF1', 'SPATC1', 'FOXP3', 'ARRB2', 'TNFRSF9',
       'RNASE6', 'DBH-AS1', 'TENT5C', 'ID3', 'APOE', 'LAX1'], dtype='<U7')

In [164]:
# final 
repeated_idx

[14784,
 17475,
 10180,
 7755,
 9165,
 8335,
 7569,
 12146,
 2771,
 10259,
 7288,
 7452,
 10303]

In [165]:
n_features = len(repeated_idx)
n_features

13

In [166]:
list(col_idx_map_list[0].keys()).index(1)

0

In [167]:
feature_idx_list = []

for col_idx_map in col_idx_map_list:
    feature_idx = []
    for rep_idx in repeated_idx:
        if rep_idx in col_idx_map.keys():
            feature_idx.append(list(col_idx_map.keys()).index(rep_idx))
    feature_idx_list.append(feature_idx)        
feature_idx_list    

[[1023, 1224, 747, 549, 671, 602, 524, 860, 165, 750, 483, 501, 753],
 [984, 1184, 720, 538, 652, 588, 512, 825, 160, 724, 469, 489, 727],
 [1065, 1273, 778, 574, 702, 628, 548, 897, 171, 782, 504, 525, 785],
 [1046, 1253, 767, 573, 691, 627, 546, 873, 167, 770, 504, 522, 772],
 [1005, 1202, 737, 546, 663, 597, 521, 844, 164, 741, 478, 498, 743],
 [818, 979, 611, 449, 552, 497, 425, 694, 138, 613, 391, 406, 615],
 [1015, 1207, 739, 547, 662, 599, 521, 849, 162, 743, 478, 499, 746],
 [1107, 1319, 787, 584, 706, 636, 558, 914, 169, 791, 516, 536, 794],
 [959, 1155, 703, 521, 635, 570, 498, 807, 160, 707, 461, 477, 709],
 [779, 939, 573, 427, 521, 472, 404, 652, 131, 575, 371, 383, 577]]

In [168]:
# feature_name_list = []

# for i, ids in enumerate(feature_idx_list ):
#     feature_name = np.array(list(col_idx_map_list[i].values()))[ids]
#     feature_name_list.append(feature_name)
# feature_name_list  

In [169]:
w_list[2]

array([[ 3.6287211e-02,  5.2281581e-02, -4.7744434e-02],
       [-4.1748736e-02,  4.0929358e-02, -3.6689967e-02],
       [-2.7881967e-02, -6.8633388e-05,  5.5764709e-02],
       ...,
       [ 6.4271945e-03,  5.3521659e-02,  2.2679424e-02],
       [-2.5195029e-02, -2.9318940e-02,  2.8358078e-02],
       [ 2.4859261e-02,  3.7713781e-02,  7.9835504e-03]], dtype=float32)

In [170]:
w_list[2][1168]

array([0.00433513, 0.00386086, 0.0276604 ], dtype=float32)

In [171]:
# (w_list[0][1037] + w_list[1][997] +w_list[2][1077])/n_features

In [172]:
# [w[idx] for idx in feature_idx for w, feature_idx in zip(w_list, feature_idx_list)]
# output: 3(==n_classes)* n_features
mean_w_list = []

for w, feature_idx in zip(w_list, feature_idx_list):
    for idx in feature_idx:
        w[idx]
        
for i in range(n_features):
    ary = np.zeros(3)
    for w, feature_idx in zip(w_list, feature_idx_list):
        for c in range(n_classes):
            ary[c] += w[feature_idx[i]][c]
    mean_w_list.append(ary/n_features)
    
mean_w_list[0]    

array([-0.00579295, -0.01111252,  0.01562402])

In [173]:
w.shape

(953, 3)

In [174]:
# # non bonf 
# mean_w_list = []
        
# for i in range(n_features):
#     ary = np.zeros(3)
#     for w in w_list:
#         for c in range(n_classes):
#             ary[c] += w[repeated_idx[i]][c]
#             print(w[repeated_idx[i]][c])
#     mean_w_list.append(ary/n_features)
    
# mean_w_list[0]    

In [175]:
mean_w_list

[array([-0.00579295, -0.01111252,  0.01562402]),
 array([-0.00887163, -0.00234678, -0.00513912]),
 array([-0.00665409,  0.00032249,  0.00379274]),
 array([-0.00599484, -0.01017522,  0.00287159]),
 array([-0.011346  , -0.00395544, -0.00405095]),
 array([ 0.01126967, -0.00390696,  0.00718204]),
 array([ 0.00629946, -0.01131399, -0.0100597 ]),
 array([-0.0116012 , -0.00639622, -0.01342672]),
 array([0.00801876, 0.00654398, 0.00347906]),
 array([-0.0042868 , -0.00840255, -0.0190197 ]),
 array([0.02370058, 0.00539297, 0.00275282]),
 array([-0.00710489, -0.00306664,  0.00227833]),
 array([ 0.00371064,  0.00591692, -0.01106254])]

In [176]:
from scipy import stats

def bonferroni_correction_function(cls0, cls1):
    # 2.8261361067148994e-06
    alpha = 0.05
    no_test = 17692
    alpha_bonferroni = alpha/no_test 

    pval_list = []
    
    # number of valid features
    counter = 0
    for i in range(n_features):
        statistic, pvalue = stats.ttest_ind(cls0[i], cls1[i], equal_var=False)
        pval_list.append(pvalue)
        
        # print(len(statistic)) #17962
        # print(len(pvalue)) #17692
        print(pvalue)

        # low p-value
        if pvalue <= alpha_bonferroni:
            counter = counter + 1

            # size (selected feature count)*(number of training data per class) array

        # break
    # class0, class1의 경우 대략 900 ~ 1300개
    # class1, class2의 경우 대략 200 ~ 300개
    # class2, class0의 경우 대략 200 ~ 400개
    # print("number of selected features per fold: ", counter)

    # save feature index without duplication
    # print("# of features w dups: ", len(features))
    
    # print("# of features w/o dups: ", len(nodup_features))

    return pval_list

# print(len(final_nodup_features)) #54

In [177]:
pval0, pval1, pval2 = [], [], []

for cls0, cls1, cls2, f_idx in zip(x_train_cls0_list, x_train_cls1_list, x_train_cls2_list, feature_idx_list):
  # cls with important features
  cls0 = cls0[:, f_idx]
  cls1 = cls1[:, f_idx]
  cls2 = cls2[:, f_idx]

#   print(cls0.T.shape) # cls0.T.shape = (20, 68==#of class 0)
#   print(cls1.T.shape)
#   print(cls2.T.shape)

  
  p0 = bonferroni_correction_function(cls0.T, cls1.T)
  # print("\ncls1, cls2")
  p1 = bonferroni_correction_function(cls1.T, cls2.T)
  # print("\ncls2, cls0")
  p2 = bonferroni_correction_function(cls2.T, cls0.T)

# len(p0)==20
# 10*20
  pval0.append(p0)
  pval1.append(p1)
  pval2.append(p2)

#   print(np.mean(p0))

1.0353968866005354e-07
3.565129176396806e-07
9.387663860408752e-08
1.454266357810666e-06
1.3995046848241816e-06
4.413525457288672e-18
3.9301789641221277e-07
2.1040446931125624e-14
5.514536345856748e-10
4.598212515610165e-14
1.0166227030174779e-07
1.282502555375715e-06
1.6799283452334203e-12
0.0053279311197000235
0.022300729406231658
0.015635593929359225
0.19968296794951843
0.007183460903552426
9.865070261556964e-07
0.003067044289117309
1.8032328418447082e-13
0.029565134857415416
0.0003616740014280471
0.6424421917093688
0.029890491353727622
0.009966767100803924
0.0005649909212557124
0.0004619498283041409
0.00010836917560051344
1.4809251494428986e-05
0.004744530962193128
2.951605836975049e-09
0.003472796021796063
0.0013887246846144372
3.1787190266058045e-07
1.0800815915666799e-07
4.2687983906541494e-07
0.0006729432165270587
4.503172813413753e-08
2.5908706952513123e-13
6.627681857285562e-09
5.256105994390456e-07
5.2750398328857135e-14
2.6846327239018316e-10
8.47892029583343e-08
8.84653003

In [178]:
x_train_cls1_list[0].shape

(67, 1241)

In [179]:
# pval0

In [180]:
# sval =1.2338236680153996e-12 + 2.220643124826812e-07+1.3737044280375801e-07+3.3140214724153804e-09+1.892502354865443e-06+3.341293646189116e-07+2.7540324673693234e-07+1.4823732618674294e-06+9.377785509554675e-11+2.451487785024761e-06

In [181]:
# sval/10

In [182]:
np.mean(pval1, axis=0)

array([0.00222051, 0.00449945, 0.05993344, 0.03053641, 0.01324413,
       0.00120027, 0.00116242, 0.00121314, 0.01537811, 0.09146796,
       0.11217737, 0.00669583, 0.0320166 ])

In [183]:
# pval1

In [185]:
print(np.mean(pval0, axis=0))
print(np.mean(np.mean(pval0, axis=0)),'\n')
print(np.mean(pval1, axis=0))
print(np.mean(np.mean(pval1, axis=0)),'\n')
print(np.mean(pval2, axis=0))
print(np.mean(np.mean(pval2, axis=0)),'\n')
print('\n')
print(np.std(pval0, axis=0))
print(np.mean(np.std(pval0, axis=0)),'\n')
print(np.std(pval1, axis=0))
print(np.mean(np.std(pval1, axis=0)),'\n')
print(np.std(pval2, axis=0))
print(np.mean(np.std(pval2, axis=0)),'\n')

[1.29317125e-07 2.77444470e-07 6.57237310e-07 3.23768093e-07
 5.84203985e-07 6.33840664e-08 3.30457813e-07 2.66760774e-07
 2.87324564e-08 3.16437962e-07 7.94301887e-07 4.82242739e-07
 5.46838685e-07]
3.6931748969573653e-07 

[0.00222051 0.00449945 0.05993344 0.03053641 0.01324413 0.00120027
 0.00116242 0.00121314 0.01537811 0.09146796 0.11217737 0.00669583
 0.0320166 ]
0.02859581799445524 

[0.00780016 0.00751554 0.03254995 0.0500428  0.00743223 0.00415087
 0.02187874 0.01614285 0.00831638 0.00539192 0.01195187 0.00648075
 0.02373268]
0.015645134926668632 



[2.02495996e-07 6.51949057e-07 6.92279971e-07 6.37698510e-07
 7.23172856e-07 9.76427080e-08 7.29389453e-07 4.50058446e-07
 7.67615643e-08 7.71356333e-07 1.07186222e-06 5.60580983e-07
 8.08027070e-07]
5.748673207971041e-07 

[0.00252007 0.00710614 0.16031848 0.05975398 0.02308021 0.00255016
 0.00153546 0.00344055 0.02871445 0.20146337 0.19900628 0.00997793
 0.07693892]
0.0597235384229695 

[0.02124659 0.01672221 0.06953595 0.108207

In [89]:
print('running time: {} sec'.format(time_delta.seconds))

running time: 131 sec


In [90]:
y_all_test = np.concatenate(y_all_test)
y_all_proba = np.concatenate(y_all_proba)

In [91]:
# roc curve
fpr = dict()
tpr = dict()
# mean_fpr, mean_tpr = dict({0:0, 1:0, 2:0}), dict({0:0, 1:0, 2:0})
roc_auc = dict()

# pr curve
pre = dict()
rec = dict()
# mean_pre, mean_rec = dict({0:0, 1:0, 2:0}), dict({0:0, 1:0, 2:0})
pr_auc = dict()

In [ ]:
# PR curve
for i, label in zip(range(3), cls):
  pre[i], rec[i], _ = precision_recall_curve(y_test[:, i], prediction[:, i])
#   mean_pre[i] /= 10
#   mean_rec[i] /= 10

  pr_auc[i] = auc(rec[i], pre[i])
  plt.plot(rec[i], pre[i], lw=2, label='PR curve of class "{0}" (area = {1:0.3f})'
          ''.format(label, pr_auc[i]))
    

plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("PR curve")

plt.savefig('PR curve.png', dpi=300)
plt.show()


In [ ]:
# ROC curve
for i, label in zip(range(3), cls):
#   mean_fpr[i] /= 10
#   mean_tpr[i] /= 10
  fpr[i], tpr[i], _ = roc_curve(y_test[:, i],
                                    prediction[:, i])
  roc_auc[i] = auc(fpr[i], tpr[i])
  plt.plot(fpr[i], tpr[i], lw=2, label='ROC curve of class "{0}" (area = {1:0.3f})'
          ''.format(label, roc_auc[i]))

plt.xlabel("false positive rate")
plt.ylabel("true positive rate")
plt.legend(loc="best")
plt.title("ROC curve")

plt.savefig('ROC curve.png', dpi=300)
plt.show()


In [ ]:
# h_loss, h_val_loss, h_acc, h_val_acc
# nsplit = 8
nsplit = float(nsplit)

h_loss[:] = [x/nsplit for x in h_loss]
h_val_loss[:] = [x/nsplit for x in h_val_loss]
h_acc[:] = [x/nsplit for x in h_acc]
h_val_acc[:] = [x/nsplit for x in h_val_acc]


plt.figure(figsize=(12,8))
plt.plot(h_loss)
plt.plot(h_val_loss)
plt.legend(['training loss','test loss'])
plt.grid()
plt.savefig('loss.png', dpi=300)
plt.show()


plt.figure(figsize=(12,8))
plt.plot(h_acc)
plt.plot(h_val_acc)
plt.xlabel('epoch')
plt.legend(['training accuracy','test accuracy'], prop={'size': 6})
# plt.grid()
plt.savefig('acc.png', dpi=300)
plt.show()

In [ ]:
acc_list, rec_list, pre_list, total_acc_list = [], [], [], []

for one_pred, y_test in zip(one_pred_list, y_test_list):
  acc, rec, pre, total_acc = result(one_pred, y_test)
  acc_list.append(acc)
  rec_list.append(rec)
  pre_list.append(pre)
  total_acc_list.append(total_acc)

In [ ]:
train_acc_list

In [ ]:
tf.print("Total TRAIN Accuracy = {:.3f}".format(K.sum(train_acc_list)/len(train_acc_list)))

In [ ]:
# total accuracy
total_acc_list

In [ ]:
# decode one hot vector
y_pred = np.argmax(one_pred, axis=1).reshape(-1, 1)
# y_pred

In [ ]:
from sklearn import metrics

for one_pred, y_test in zip(one_pred_list, y_test_list):
  
  dec_y_pred = np.argmax(one_pred, axis=1).reshape(-1, 1)
  dec_y_test = np.argmax(y_test, axis=1).reshape(-1, 1)

  # Print the confusion matrix
  # print(metrics.confusion_matrix(dec_y_test, dec_y_pred))

  # Print the precision and recall, among other metrics
  print(metrics.classification_report(dec_y_test, dec_y_pred, digits=3))

In [ ]:
# number of class
for i in range(y_test_list[0].shape[1]):
  acc_mean, rec_mean, pre_mean = 0, 0, 0
  
  # k-fold
  for j in range(len(acc_list)):  
    acc_mean += acc_list[j][i]
    rec_mean += rec_list[j][i]
    pre_mean += pre_list[j][i]
  
  acc_mean /= len(acc_list)
  rec_mean /= len(rec_list)
  pre_mean /= len(pre_list)

  print("\nclass: ", i)
  # mean score of k-folds
  tf.print("Accuracy = {:.3f}".format(acc_mean))
  tf.print("Recall = {:.3f}".format(rec_mean))
  tf.print("Precision = {:.3f} ".format(pre_mean))

In [ ]:
len(acc_list)

In [187]:
lst = [np.array([-0.00579295, -0.01111252,  0.01562402]),
 np.array([-0.00887163, -0.00234678, -0.00513912]),
 np.array([-0.00665409,  0.00032249,  0.00379274]),
 np.array([-0.00599484, -0.01017522,  0.00287159]),
 np.array([-0.011346  , -0.00395544, -0.00405095]),
 np.array([ 0.01126967, -0.00390696,  0.00718204]),
 np.array([ 0.00629946, -0.01131399, -0.0100597 ]),
 np.array([-0.0116012 , -0.00639622, -0.01342672]),
 np.array([0.00801876, 0.00654398, 0.00347906]),
 np.array([-0.0042868 , -0.00840255, -0.0190197 ]),
 np.array([0.02370058, 0.00539297, 0.00275282]),
 np.array([-0.00710489, -0.00306664,  0.00227833]),
 np.array([ 0.00371064,  0.00591692, -0.01106254])
]

In [193]:
np.mean(lst, axis=0)

array([-0.00066564, -0.00326923, -0.00190601])

In [198]:
lst = [0.2408451, 0.21637085, 0.32011274, 0.28361083, 0.23472572, 0.34361835,
 0.27995798, 0.09976171, 0.28402286, 0.26182375, 0.28349403, 0.26317557,
 0.23039316, 0.34074784, 0.34802407, 0.29737508, 0.296906, 0.28480688]


np.mean(lst)

0.27276513999999996

# **SVM**

In [ ]:
acc_list, rec_list, pre_list, total_acc_list = [], [], [], []
y_all_test,y_all_proba = [], []

In [ ]:
# Support Vector Classifier
# one vs rest

from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import precision_recall_curve, roc_curve, auc

# c_list = [0.1, 1.0, 10, 100, 1000]
c_list = [1.0, 10, 100, 1000]
cls = ['desert', 'inflamed', 'excluded']

with tf.device('/gpu:0'):
#     for c in c_list:
    for x_train, decoded_y_train, x_test, y_test in zip(x_train_list, decoded_y_train_list, x_test_list, y_test_list):
      svm = OneVsRestClassifier(SVC(C=100, gamma='scale', probability=True)).fit(x_train, decoded_y_train)
      prob_y_pred_svc = svm.predict_proba(x_test)
      y_pred_svc = svm.predict(x_test)

      y_all_test.append(y_test)
      y_all_proba.append(prob_y_pred_svc)

      # print(y_pred_svc)

      # encode y_pred_svc to one-hot vector 
      Y = encoder.fit_transform(y_pred_svc)
      y_pred_svc = pd.get_dummies(Y).values.astype(np.float32)

      # print(y_pred_svc)



      acc = accuracy_score(y_test, y_pred_svc)
      print(round(acc, 3))
      total_acc_list.append(round(acc, 3))

      acc, rec, pre, total_acc = result(y_pred_svc, y_test)
      acc_list.append(acc)
      rec_list.append(rec)
      pre_list.append(pre)
      total_acc_list.append(total_acc)
    # print(round(sum(total_acc_list)/len(total_acc_list), 3))
    print(sum(total_acc_list)/len(total_acc_list))

In [ ]:
sum(total_acc_list)/len(total_acc_list)

In [ ]:
# number of class
for i in range(y_test_list[0].shape[1]):
  acc_mean, rec_mean, pre_mean = 0, 0, 0
  
  # k-fold
  for j in range(len(acc_list)):  
    acc_mean += acc_list[j][i]
    rec_mean += rec_list[j][i]
    pre_mean += pre_list[j][i]
  
  acc_mean /= len(acc_list)
  rec_mean /= len(rec_list)
  pre_mean /= len(pre_list)

  print("\nclass: ", i)
  # mean score of k-folds
  tf.print("Accuracy = {:.3f}".format(acc_mean))
  tf.print("Recall = {:.3f}".format(rec_mean))
  tf.print("Precision = {:.3f} ".format(pre_mean))

In [ ]:
y_all_test = np.concatenate(y_all_test)
y_all_proba = np.concatenate(y_all_proba)

In [ ]:
# PR curve
pr_auc = dict()
for i, label in zip(range(3), cls):
  pre[i], rec[i], _ = precision_recall_curve(y_all_test[:, i], y_all_proba[:, i])
#   mean_pre[i] /= 10
#   mean_rec[i] /= 10

  pr_auc[i] = auc(rec[i], pre[i])
  plt.plot(rec[i], pre[i], lw=2, label='PR curve of class "{0}" (area = {1:0.3f})'
          ''.format(label, pr_auc[i]))
    

plt.xlabel("recall")
plt.ylabel("precision")
plt.legend(loc="best")
plt.title("PR curve")

plt.savefig('PR curve.png', dpi=300)
plt.show()


In [ ]:
# ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()
for i, label in zip(range(3), cls):
#   mean_fpr[i] /= 10
#   mean_tpr[i] /= 10
  fpr[i], tpr[i], _ = roc_curve(y_all_test[:, i], y_all_proba[:, i])
  roc_auc[i] = auc(fpr[i], tpr[i])
  plt.plot(fpr[i], tpr[i], lw=2, label='ROC curve of class "{0}" (area = {1:0.3f})'
          ''.format(label, roc_auc[i]))

plt.xlabel("false positive rate")
plt.ylabel("true positive rate")
plt.legend(loc="best")
plt.title("ROC curve")

plt.savefig('ROC curve.png', dpi=300)
plt.show()

In [ ]:
# output개수가 0인 class가 있을 때 add [0]*y_pred_svc.shape[0]

# new_column = [0] * y_pred_svc.shape[0]
# y_pred_svc = np.insert(y_pred_svc, 2, new_column, axis=1)
# y_pred_svc